In [1]:
import arcpy
from arcpy.sa import *
import mysql.connector
from arcpy import env
import numpy as np
import pickle
from dotenv import load_dotenv
import os
import pandas as pd
from tqdm.notebook import tqdm
from sqlalchemy import create_engine

In [2]:
# Load environment variables
load_dotenv()
HOST = os.environ.get('HOST')
USER= os.environ.get('USER_NAME')
PASSWD = os.environ.get('PASSWD')
PORT = os.environ.get('PORT')
DATABASE = 'dy_database'

In [2]:
# raster list
raster_path = r'D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb'
env.workspace = raster_path
raster_list = arcpy.ListRasters()

In [3]:
# 将栅格数据并转换为NumPy数组并序列化为二进制格式
def read_numpy_array(in_raster):
    array = arcpy.RasterToNumPyArray(in_raster, nodata_to_value=0)
    data_dict = {}
    for one_row in range(array.shape[0]):
        data_dict[one_row]=pickle.dumps(array[one_row])
    return data_dict

# 将序列化二进制格式栅格转换为数组
def bin_to_array(in_binary):
    array = pickle.loads(in_binary)
    return array

# 添加新列

def add_columns(cursor,conn,table_name,col_name,new_col_type):
    # 检查该列是否存在
    cursor.execute(f"SHOW COLUMNS FROM {table_name} LIKE '{col_name}'")
    result = cursor.fetchone()
    # 如果列不存在，则添加列
    if not result:
        cursor.execute(f"ALTER TABLE {table_name} ADD {col_name} {new_col_type}")
        conn.commit()
        print(f"Column '{col_name}' added to '{table_name}'.")
    else:
        print(f"Column '{col_name}' already exists in '{table_name}'.")
# 数组重塑
def represhape(arr, new_shape_cols):
    # 按照给定的行数，将数组重塑为二维数组
    size = arr.size
    # 计算列数
    new_shape_rows = math.ceil(size / new_shape_cols)
    # 计算新size
    new_size = new_shape_rows * new_shape_cols
    # 将旧数组的值拷贝到新数组中
    new_arr = np.full(new_size,None)
    new_arr[:size] = arr
    return new_arr.reshape(new_shape_rows, new_shape_cols)
# 读取npy文件
def read_npy_file(file_path):
    return np.load(file_path)

# 写入npy文件
def write_npy_file(file_path, arr):
    np.save(file_path, arr)
    return True


In [5]:
# 连接到MySQL数据库
# conn = mysql.connector.connect(host=HOST, user=USER, password=PASSWD, database=DATABASE)
engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                       format(USER, PASSWD, HOST, DATABASE))

In [ ]:

for one_raster in tqdm(raster_list):
    print(one_raster)
    # 创建一个示例 DataFrame
    table_name = one_raster.lower()
    data = arcpy.RasterToNumPyArray(one_raster).flatten()
    data_array = represhape(data,344)
    df = pd.DataFrame(data_array)
    chunk_size = 10
    # 将DataFrame中的数据写入到MySQL数据库中
    try:
        print("Writing data to MySQL database...")
        df.to_sql(name=table_name, con=engine, if_exists='replace', index=False, chunksize=chunk_size)
        print("Writing data to MySQL database...OK")
    except Exception as e:
        print(f"Error occurred when writing to table {table_name}: {e}")


In [4]:
# 转存为npy文件
save_path = r"F:\ArcgisData\npy_path\dy"
for one_raster in tqdm(raster_list):
    data = arcpy.RasterToNumPyArray(one_raster, nodata_to_value=0)
    np.save(os.path.join(save_path, f"{one_raster.lower()}.npy"), data)


  0%|          | 0/38 [00:00<?, ?it/s]